In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import bz2
import re
import matplotlib

from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
!apt-get install libbz2-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [ ]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('./drive/My Drive/Colab Notebooks/train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('./drive/My Drive/Colab Notebooks/test.ft.txt.bz2')

In [ ]:
!pwd

/content


**bold text**# Compiling texts

In [ ]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, random_state=57643892, test_size=0.2)

In [ ]:
MAX_FEATURES = 12000
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
val_texts = tokenizer.texts_to_sequences(val_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)

In [ ]:
MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)
train_texts = pad_sequences(train_texts, maxlen=MAX_LENGTH)
val_texts = pad_sequences(val_texts, maxlen=MAX_LENGTH)
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)

In [ ]:
def build_rnn_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.CuDNNGRU(128, return_sequences=True)(embedded)
    x = layers.CuDNNGRU(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
rnn_model = build_rnn_model()

In [ ]:
rnn_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 255)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 255, 64)           768000    
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (None, 255, 128)          74496     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 128)               99072     
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               3300      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [ ]:
rnn_model.fit(
    train_texts, 
    train_labels, 
    batch_size=128,
    epochs=8,
    validation_split = 0.1,
    validation_data=(val_texts, val_labels))

Epoch 1/8
20250/20250 [==============================] - 784s 39ms/step - loss: 0.1314 - binary_accuracy: 0.9532 - val_loss: 0.1375 - val_binary_accuracy: 0.9500
Epoch 2/8
20250/20250 [==============================] - 781s 39ms/step - loss: 0.1337 - binary_accuracy: 0.9523 - val_loss: 0.1462 - val_binary_accuracy: 0.9476
Epoch 3/8
20250/20250 [==============================] - 781s 39ms/step - loss: 0.1356 - binary_accuracy: 0.9515 - val_loss: 0.1413 - val_binary_accuracy: 0.9485
Epoch 4/8
20250/20250 [==============================] - 776s 38ms/step - loss: 0.1382 - binary_accuracy: 0.9504 - val_loss: 0.1453 - val_binary_accuracy: 0.9472
Epoch 5/8
20250/20250 [==============================] - 770s 38ms/step - loss: 0.1400 - binary_accuracy: 0.9496 - val_loss: 0.1468 - val_binary_accuracy: 0.9462
Epoch 6/8
20250/20250 [==============================] - 766s 38ms/step - loss: 0.1457 - binary_accuracy: 0.9474 - val_loss: 0.1525 - val_binary_accuracy: 0.9443
Epoch 7/8
20250/20250 [=====

In [ ]:
preds = rnn_model.predict(test_texts)
print('Accuracy score: {:0.4}'.format(accuracy_score(test_labels, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(test_labels, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_labels, preds)))

Accuracy score: 0.9433
F1 score: 0.9433
ROC AUC score: 0.9848


# New Section